# Data Export for (https://github.com/albblgb/Deep-Steganalysis.git)

In [9]:
import os
import sys

import shutil

import numpy as np
module_path = os.path.abspath(os.path.join('..', '..', '..', '..'))
print('added path:', module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

from model_xray.zenml.pipelines.data_creation.dataset_compilation import retreive_datasets
from model_xray.config_classes import *
from PIL import Image

from model_xray.zenml.pipelines.data_creation.dataset_compilation import compile_and_save_preprocessed_images_dataset_pipeline
from model_xray.options import *
from model_xray.config_classes import *

curr_mc = 'famous_le_10m'

train_mzs, test_mzs = dataset_split[curr_mc]

imsize = 256
imtype = ImageType.GRAYSCALE_THREEPART_WEIGHTED_AVG

image_rep_config = ImageRepConfig.ret_image_rep_config(imtype)

image_preprocess_config=ImagePreprocessConfig(
    image_height=imsize,
    image_width=imsize,
)

from model_xray.utils.dataset_utils import *

pretrained_model_train_configs = get_pretrained_model_configs(train_mzs)
pretrained_model_test_configs = get_pretrained_model_configs(test_mzs)

x_train = 15

test_benigns_dataset_name = get_dataset_name(
    mc=curr_mc,
    xs=[None,],
    imsize=imsize,
    imtype=imtype,
    ds_type='test'
)

test_attacked_dataset_name = get_dataset_name(
    mc=curr_mc,
    xs=[x_train,],
    imsize=imsize,
    imtype=imtype,
    ds_type='test'
)

trainset_name = get_dataset_name(
    mc=curr_mc,
    xs=[None,x_train],
    imsize=imsize,
    imtype=imtype,
    ds_type='train'
)



trainset_names = [trainset_name,]

testset_names = [
    test_benigns_dataset_name, test_attacked_dataset_name
]

requested_datasets = trainset_names + testset_names

datasets = retreive_datasets(requested_datasets)

def output_imgs(img_arrs, path, output_as_rgb=False):
    os.makedirs(path, exist_ok=True)
    for i, img_arr in enumerate(img_arrs):
        img = Image.fromarray(img_arr)
        if output_as_rgb:
            img = img.convert('RGB')
        img.save(f'{path}/{i+1}.png')

def output_dataset(X,y, benign_path, mal_path, reset_dirs=False):
    if reset_dirs:
        if os.path.exists(benign_path):
            shutil.rmtree(benign_path)
        if os.path.exists(mal_path):
            shutil.rmtree(mal_path)

    os.makedirs(benign_path, exist_ok=True)
    os.makedirs(mal_path, exist_ok=True)

    X_benign = X[y == PreprocessedImageDatasetLabel.BENIGN]
    output_imgs(X_benign, benign_path)
    X_mal = X[y == PreprocessedImageDatasetLabel.ATTACKED]
    output_imgs(X_mal, mal_path)

def merge_datasets(datasets):
    Xs = []
    ys = []
    for X,y in datasets:
        Xs.append(X)
        ys.append(y)
    return np.vstack(Xs), np.hstack(ys)

train_datasets = [datasets[name] for name in trainset_names]
test_datasets = [datasets[name] for name in testset_names]
X_train, y_train = merge_datasets(train_datasets)
X_test, y_test = merge_datasets(test_datasets)

def fill_deep_steganalysis_data_dirs(*, X_train, y_train, X_test, y_test, root_dir='', X_val=None, y_val=None):
    train_path = f'{root_dir}/train_data_dir'
    test_path = f'{root_dir}/test_data_dir'
    val_path = f'{root_dir}/val_data_dir'

    output_dataset(X_train, y_train, f'{train_path}/cover', f'{train_path}/stego', reset_dirs=True)
    output_dataset(X_test, y_test, f'{test_path}/cover', f'{test_path}/stego', reset_dirs=True)
    if X_val is None or y_val is None:
        X_val, y_val = X_test, y_test
    
    output_dataset(X_val, y_val, f'{val_path}/cover', f'{val_path}/stego', reset_dirs=True)

# root_dir = '/mnt/exdisk2/image_steganalysis/deep_steganalysis'
fill_deep_steganalysis_data_dirs(X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, root_dir='/mnt/exdisk2/image_steganalysis/deep_steganalysis')

added path: /home/danielg/danigil/ModelXRay


# Compile registry

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..', '..', '..', '..'))
print('added path:', module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

from model_xray.zenml.pipelines.data_creation.dataset_compilation import compile_preprocessed_images_registry_pipeline
from model_xray.options import *
from model_xray.config_classes import *

model_names = model_collections['famous_le_100m'].union(model_collections['famous_le_10m'])
pretrained_model_configs = [PretrainedModelConfig(name=model_name, repo=ModelRepos.KERAS) for model_name in model_names]
compile_preprocessed_images_registry_pipeline(pretrained_model_configs=pretrained_model_configs)

added path: /home/danielg/danigil/ModelXRay


PyTorch version 2.4.0+cu118 available.
TensorFlow version 2.14.0 available.
Using an external artifact as step input currently invalidates caching for the step and all downstream steps. Future releases will introduce hashing of artifacts which will improve this behavior.
Initiating a new run for the pipeline: compile_preprocessed_images_registry_pipeline.
Uploading external artifact to 'external_artifacts/external_e85c68d0-9445-41cf-b287-3ff1f6b6998c'.
No materializer is registered for type <class 'model_xray.config_classes.PretrainedModelConfig'>, so the default Pickle materializer was used. Pickle is not production ready and should only be used for prototyping as the artifacts cannot be loaded when running with a different Python version. Please consider implementing a custom materializer for type <class 'model_xray.config_classes.PretrainedModelConfig'> according to the instructions at https://docs.zenml.io/how-to/handle-data-artifacts/handle-custom-data-types
No materializer is reg

PipelineRunResponse(body=PipelineRunResponseBody(created=datetime.datetime(2024, 9, 5, 13, 46, 44, 959530), updated=datetime.datetime(2024, 9, 5, 14, 32, 4, 982059), user=UserResponse(body=UserResponseBody(created=datetime.datetime(2024, 8, 17, 23, 59, 37, 668468), updated=datetime.datetime(2024, 8, 18, 0, 2, 49, 626363), active=True, activation_token=None, full_name='', email_opted_in=False, is_service_account=False, is_admin=True), metadata=None, resources=None, id=UUID('8c66513b-5e16-4bbf-a271-7b389d6c1d2d'), permission_denied=False, name='default'), status=<ExecutionStatus.COMPLETED: 'completed'>, stack=StackResponse(body=StackResponseBody(created=datetime.datetime(2024, 8, 18, 0, 9, 44, 367644), updated=datetime.datetime(2024, 8, 20, 0, 9, 7, 782296), user=UserResponse(body=UserResponseBody(created=datetime.datetime(2024, 8, 17, 23, 59, 37, 668468), updated=datetime.datetime(2024, 8, 18, 0, 2, 49, 626363), active=True, activation_token=None, full_name='', email_opted_in=False, is_

# Compile datasets

In [7]:
from typing import Dict, List, Set
import os
import sys

import shutil

import numpy as np
module_path = os.path.abspath(os.path.join('..', '..', '..', '..'))
print('added path:', module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

from model_xray.zenml.pipelines.data_creation.dataset_compilation import compile_and_save_preprocessed_images_dataset_pipeline
from model_xray.options import *
from model_xray.config_classes import *

curr_mc = 'famous_le_10m'

train_mzs, test_mzs = dataset_split[curr_mc]

# imsize = 256
# imtype = ImageType.GRAYSCALE_THREEPART_WEIGHTED_AVG
imsize = 100
imtype = ImageType.GRAYSCALE_FOURPART

image_rep_config = ImageRepConfig.ret_image_rep_config(imtype)

image_preprocess_config=ImagePreprocessConfig(
    image_height=imsize,
    image_width=imsize,
)

from model_xray.utils.dataset_utils import *

added path: /home/danielg/danigil/ModelXRay


In [2]:
get_dataset_name(mc="famous_le_10m_zoos", xs=[None,15, 33, 3, 11, 30], imsize=256, imtype=ImageType.GRAYSCALE_FOURPART, ds_type='train')

'ds_type:train,imsize:256,imtype:grayscale_fourpart,mc:famous_le_10m_zoos,xs:0|3|11|15|30|33'

In [8]:
pretrained_model_train_configs = get_pretrained_model_configs(train_mzs)
pretrained_model_test_configs = get_pretrained_model_configs(test_mzs)

test_benigns_dataset_name = get_dataset_name(
    mc=curr_mc,
    xs=[None,],
    imsize=imsize,
    imtype=imtype,
    ds_type='test'
)

compile_and_save_preprocessed_images_dataset_pipeline(
    dataset_name=test_benigns_dataset_name,
    pretrained_model_configs=pretrained_model_test_configs,
    x_values=[None,],
    image_rep_config=image_rep_config,
    image_preprocess_config=image_preprocess_config,
)

for x in range(1, 24):
    xs = [None, x]
    train_dataset_name = get_dataset_name(
        mc=curr_mc,
        xs=xs,
        imsize=imsize,
        imtype=imtype,
        ds_type='train'
    )

    compile_and_save_preprocessed_images_dataset_pipeline(
        dataset_name=train_dataset_name,
        pretrained_model_configs=pretrained_model_train_configs,
        # x_values=[None,] + [x for x in range(1, 24)],
        x_values=xs,
        image_rep_config=image_rep_config,
        image_preprocess_config=image_preprocess_config,
    )
    xs = [x,]

    test_dataset_name = get_dataset_name(
        mc=curr_mc,
        xs=xs,
        imsize=imsize,
        imtype=imtype,
        ds_type='test'
    )
    compile_and_save_preprocessed_images_dataset_pipeline(
        dataset_name=test_dataset_name,
        pretrained_model_configs=pretrained_model_test_configs,
        # x_values=[None,] + [x for x in range(1, 24)],
        x_values=xs,
        image_rep_config=image_rep_config,
        image_preprocess_config=image_preprocess_config,
    )


Using an external artifact as step input currently invalidates caching for the step and all downstream steps. Future releases will introduce hashing of artifacts which will improve this behavior.
Initiating a new run for the pipeline: compile_and_save_preprocessed_images_dataset_pipeline.


Uploading external artifact to 'external_artifacts/external_21ab0325-8f22-43d7-b8b8-d32c39f7804c'.
No materializer is registered for type <class 'model_xray.config_classes.PretrainedModelConfig'>, so the default Pickle materializer was used. Pickle is not production ready and should only be used for prototyping as the artifacts cannot be loaded when running with a different Python version. Please consider implementing a custom materializer for type <class 'model_xray.config_classes.PretrainedModelConfig'> according to the instructions at https://docs.zenml.io/how-to/handle-data-artifacts/handle-custom-data-types
No materializer is registered for type <class 'model_xray.config_classes.PretrainedModelConfig'>, so the default Pickle materializer was used. Pickle is not production ready and should only be used for prototyping as the artifacts cannot be loaded when running with a different Python version. Please consider implementing a custom materializer for type <class 'model_xray.config_